In [6]:
# 코드 7-3 3개의 출력을 가진 함수형 API 구현하기

from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

post_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size,256)(post_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 출력 층에 이름 붙이기
age_prediction  = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(post_input, [age_prediction, income_prediction, gender_prediction])

나이 예측은 스칼라 회귀 문제이지만 성별 예측은 이진 클래스 문제라 훈련 방식이 다릅니다.
경사 하강법은 하나의 스칼라 값을 최소화하기 때문에 모델을 훈련하려면 이 손실들을 하나의 값으로 합쳐야 합니다.

In [7]:
# 코드 7-4 다중 출력 모델의 컴파일 옵션: 다중 손실

# 1. 리스트로 손실함수 지정
model.compile(optimizer='rmsprop', loss=['mse','categorical_crossentropy', 'binary_crossentropy'])

# 2. 딕셔너리로 손실함수 지정
model.compile(optimizer='rmsprop', loss={'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'})

솔실 값이 많이 불균형하면 모델이 개별 손실이 가장 큰 작업에 치우쳐 표현을 최적화할 것입니다.
그 결과 다른 작업들은 손해를 입습니다. 
이를 해결하기 위해 손실 값이 최종 손실에 기여하는 수준을 지정할 수 있습니다.
특히 손실 값의 스케일이 다를 때 유용합니다.

In [8]:
# 코드 7-5 다중 출력 모델의 컴파일 옵션: 손실 가중치
# 1. 리스트로 손실함수 지정 + 가중치
model.compile(optimizer='rmsprop', loss=['mse','categorical_crossentropy', 'binary_crossentropy'], loss_weights=[0.25, 1., 10.])

# 2. 딕셔너리로 손실함수 지정 + 가중치
model.compile(optimizer='rmsprop', loss={'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'}, loss_weights={'age':0.25, 'income':1., 'gender':10.})

In [9]:
model.fit(posts, [age_targets, income_targets, gender_targets],  epochs = 10, batch_size = 64)
# age_targets, income_targets, gender_targets가 넘파이 배열이라고 가정합니다.

model.fit(posts, {'age': age_targets, 'income': income_targets, 'gender': gender_targets}, epochs = 10, batch_size = 64)
# 위와 동일합니다.(출력 층에 이름을 지정했을 때만 사용할 수 있습니다.)

NameError: name 'posts' is not defined